# mRNN 101 with GenCode 25 and Hendrix-like prep

In [1]:
#from google.colab import drive
#PATH='/content/drive/'
#drive.mount(PATH)
#DATAPATH=PATH+'My Drive/data/'
#PC_FILENAME = DATAPATH+'pcRNA.fasta'
#NC_FILENAME = DATAPATH+'ncRNA.fasta'
# LOCAL
PCT_FILENAME = 'train.gc25.pcRNA.fasta'
NCT_FILENAME = 'train.gc25.ncRNA.fasta'
PCV_FILENAME = 'valid.gc25.pcRNA.fasta'
NCV_FILENAME = 'valid.gc25.ncRNA.fasta'

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import ShuffleSplit
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import GRU
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import time

tf.keras.backend.set_floatx('float32')

EPOCHS=100
SPLITS=1
K=1
EMBED_DIMEN=16
FILENAME='HDX102'

## Load and partition sequences

In [3]:
# Assume file was preprocessed to contain one line per seq.
# Prefer Pandas dataframe but df does not support append.
# For conversion to tensor, must avoid python lists.
def load_fasta(filename,label):
    DEFLINE='>'
    labels=[]
    seqs=[]
    lens=[]
    nums=[]
    num=0
    with open (filename,'r') as infile:
        for line in infile:
            if line[0]!=DEFLINE:
                seq=line.rstrip()
                num += 1   # first seqnum is 1
                seqlen=len(seq)
                nums.append(num)
                labels.append(label)
                seqs.append(seq)
                lens.append(seqlen)
    df1=pd.DataFrame(nums,columns=['seqnum'])
    df2=pd.DataFrame(labels,columns=['class'])
    df3=pd.DataFrame(seqs,columns=['sequence'])
    df4=pd.DataFrame(lens,columns=['seqlen'])
    df=pd.concat((df1,df2,df3,df4),axis=1)
    return df

# Split into train/test stratified by sequence length.
def sizebin(df):
    return pd.cut(df["seqlen"],
                              bins=[0,1000,2000,4000,8000,16000,np.inf],
                              labels=[0,1,2,3,4,5])
def make_train_test(data):
    bin_labels= sizebin(data)
    from sklearn.model_selection import StratifiedShuffleSplit
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=37863)
    # split(x,y) expects that y is the labels. 
    # Trick: Instead of y, give it it the bin labels that we generated.
    for train_index,test_index in splitter.split(data,bin_labels):
        train_set = data.iloc[train_index]
        test_set = data.iloc[test_index]
    return (train_set,test_set)

def separate_X_and_y(data):
    y=   data[['class']].copy()
    X=   data.drop(columns=['class','seqnum','seqlen'])
    return (X,y)

def make_slice(data_set,min_len,max_len):
    print("original "+str(data_set.shape))
    too_short = data_set[ data_set['seqlen'] < min_len ].index
    no_short=data_set.drop(too_short)
    print("no short "+str(no_short.shape))
    too_long = no_short[ no_short['seqlen'] > max_len ].index  # strict
    no_long_no_short=no_short.drop(too_long)
    print("no long, no short "+str(no_long_no_short.shape))
    return no_long_no_short


In [4]:
def make_kmer_table(K):
    npad='N'*K
    shorter_kmers=['']
    for i in range(K):
        longer_kmers=[]
        for mer in shorter_kmers:
            longer_kmers.append(mer+'A')
            longer_kmers.append(mer+'C')
            longer_kmers.append(mer+'G')
            longer_kmers.append(mer+'T')
        shorter_kmers = longer_kmers
    all_kmers = shorter_kmers
    kmer_dict = {}
    kmer_dict[npad]=0
    value=1
    for mer in all_kmers:
        kmer_dict[mer]=value
        value += 1
    return kmer_dict

KMER_TABLE=make_kmer_table(K)

def strings_to_vectors(data,uniform_len):
    all_seqs=[]
    for seq in data['sequence']:
        i=0
        seqlen=len(seq)
        kmers=[]
        while i < seqlen-K+1:
            kmer=seq[i:i+K]
            i += 1
            value=KMER_TABLE[kmer]
            kmers.append(value)
        pad_val=0
        while i < uniform_len:
            kmers.append(pad_val)
            i += 1
        all_seqs.append(kmers)
    pd2d=pd.DataFrame(all_seqs)
    return pd2d   # return 2D dataframe, uniform dimensions

In [5]:
def build_model(maxlen,dimen):
    vocabulary_size=4**K+1   # e.g. K=3 => 64 DNA K-mers + 'NNN'
    act="sigmoid"
    dt='float32'

    neurons=16
    rnn = keras.models.Sequential()
    embed_layer = keras.layers.Embedding(
        vocabulary_size,EMBED_DIMEN,input_length=maxlen);
    rnn1_layer = keras.layers.Bidirectional(
        keras.layers.GRU(neurons, return_sequences=True, dropout=0.50, 
            input_shape=[maxlen,dimen]))
    rnn2_layer = keras.layers.Bidirectional(
        keras.layers.GRU(neurons, dropout=0.50, return_sequences=True))
    dense1_layer = keras.layers.Dense(neurons,activation=act,dtype=dt)
    dense2_layer = keras.layers.Dense(neurons,activation=act,dtype=dt)
    output_layer = keras.layers.Dense(1,activation=act,dtype=dt)

    rnn.add(embed_layer)
    rnn.add(rnn1_layer)
    rnn.add(rnn2_layer)
    rnn.add(dense1_layer)
    rnn.add(dense2_layer)
    rnn.add(output_layer)

    bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
    print("COMPILE")
    rnn.compile(loss=bc, optimizer="Adam",metrics=["accuracy"])
    return rnn

In [6]:
def train_and_validate(X,y,Xvalid,yvalid,eps,maxlen,dimen):
    cv_scores = []
    fold=0
    rnn2=None
    X_train=X
    y_train=y
    X_valid=Xvalid
    y_valid=yvalid

    print("BUILD MODEL")
    rnn2=build_model(maxlen,dimen)

    print("FIT")
    # this is complaining about string to float
    start_time=time.time()
    history=rnn2.fit(X_train, y_train, # batch_size=10, default=32 works nicely
            epochs=eps, verbose=1,  # verbose=1 for ascii art, verbose=0 for none
            validation_data=(X_valid,y_valid) )
    end_time=time.time()
    elapsed_time=(end_time-start_time)

    fold += 1
    print("Fold %d, %d epochs, %d sec"%(fold,eps,elapsed_time))

    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

    scores = rnn2.evaluate(X_valid, y_valid, verbose=0)
    print("%s: %.2f%%" % (rnn2.metrics_names[1], scores[1]*100))
    # What are the other metrics_names?
    # Try this from Geron page 505:
    # np.mean(keras.losses.mean_squared_error(y_valid,y_pred))
    cv_scores.append(scores[1] * 100)
    print()
    print("Validation core mean %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))
    return rnn2

In [7]:
def make_kmers(MINLEN,MAXLEN,train_set):
    (X_train_all,y_train_all)=separate_X_and_y(train_set)

    # The returned values are Pandas dataframes.
    # print(X_train_all.shape,y_train_all.shape)
    # (X_train_all,y_train_all)
    # y: Pandas dataframe to Python list.
    # y_train_all=y_train_all.values.tolist()
    # The sequences lengths are bounded but not uniform.
    X_train_all
    print(type(X_train_all))
    print(X_train_all.shape)
    print(X_train_all.iloc[0])
    print(len(X_train_all.iloc[0]['sequence']))

    # X: List of string to List of uniform-length ordered lists of K-mers.
    X_train_kmers=strings_to_vectors(X_train_all,MAXLEN)
    # X: true 2D array (no more lists)
    X_train_kmers.shape

    print("transform...")
    # From pandas dataframe to numpy to list to numpy
    print(type(X_train_kmers))
    num_seqs=len(X_train_kmers)
    tmp_seqs=[]
    for i in range(num_seqs):
        kmer_sequence=X_train_kmers.iloc[i]
        tmp_seqs.append(kmer_sequence)
    X_train_kmers=np.array(tmp_seqs)
    tmp_seqs=None
    print(type(X_train_kmers))
    print(X_train_kmers)

    labels=y_train_all.to_numpy()
    return (X_train_kmers,labels)

In [8]:
print("Load data from files.")
nct_seq=load_fasta(NCT_FILENAME,0)
pct_seq=load_fasta(PCT_FILENAME,1)
ncv_seq=load_fasta(NCV_FILENAME,0)
pcv_seq=load_fasta(PCV_FILENAME,1)
all_train=pd.concat((nct_seq,pct_seq),axis=0)
all_valid=pd.concat((ncv_seq,pcv_seq),axis=0)
nct_seq=None
pct_seq=None
ncv_seq=None
pcv_seq=None

##print("Put aside the test portion.")
##(train_set,test_set)=make_train_test(all_seq)

print("FASTA LOADED")


Load data from files.
FASTA LOADED


## Len 200-1Kb


In [ ]:
MINLEN=200
MAXLEN=1000

print("Working on full training set, slice by sequence length.")
print("Slice size range [%d - %d)"%(MINLEN,MAXLEN))
train_subset=make_slice(all_train,MINLEN,MAXLEN)# One array to two: X and y
valid_subset=make_slice(all_valid,MINLEN,MAXLEN)# One array to two: X and y

print ("Sequence to Kmer")
(X_train,y_train)=make_kmers(MINLEN,MAXLEN,train_subset)
(X_valid,y_valid)=make_kmers(MINLEN,MAXLEN,valid_subset)
print ("Compile the model")
model=build_model(MAXLEN,EMBED_DIMEN)
print(model.summary())  # Print this only once
print ("Train and validate")
model1=train_and_validate(X_train,y_train,X_valid,y_valid,EPOCHS,MAXLEN,EMBED_DIMEN)


Working on full training set, slice by sequence length.
Slice size range [200 - 1000)
original (32000, 4)
no short (32000, 4)
no long, no short (32000, 4)
original (1000, 4)
no short (1000, 4)
no long, no short (1000, 4)
Sequence to Kmer
<class 'pandas.core.frame.DataFrame'>
(32000, 1)
sequence    GTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCC...
Name: 0, dtype: object
712
transform...
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[3 4 3 ... 0 0 0]
 [4 2 1 ... 0 0 0]
 [3 3 3 ... 0 0 0]
 ...
 [1 4 3 ... 0 0 0]
 [1 4 3 ... 0 0 0]
 [1 4 3 ... 0 0 0]]
<class 'pandas.core.frame.DataFrame'>
(1000, 1)
sequence    CTCTGTTCACGGAAAGGGCCTGGGAGCTGCAACACCTCAACAGCAA...
Name: 0, dtype: object
693
transform...
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[2 4 2 ... 0 0 0]
 [3 3 4 ... 0 0 0]
 [2 1 2 ... 0 0 0]
 ...
 [4 4 4 ... 0 0 0]
 [3 1 2 ... 0 0 0]
 [3 4 3 ... 0 0 0]]
Compile the model
COMPILE
Model: "sequential"
___________________________________________________

1000/1000 [==============================] - 712s 712ms/step - loss: 0.4084 - accuracy: 0.8131 - val_loss: 0.4268 - val_accuracy: 0.7974
Epoch 42/100
1000/1000 [==============================] - 722s 722ms/step - loss: 0.4073 - accuracy: 0.8137 - val_loss: 0.4044 - val_accuracy: 0.8091
Epoch 43/100
1000/1000 [==============================] - 728s 728ms/step - loss: 0.3993 - accuracy: 0.8191 - val_loss: 0.4147 - val_accuracy: 0.8071
Epoch 44/100
1000/1000 [==============================] - 729s 729ms/step - loss: 0.3914 - accuracy: 0.8232 - val_loss: 0.3865 - val_accuracy: 0.8266
Epoch 45/100
1000/1000 [==============================] - 727s 727ms/step - loss: 0.3950 - accuracy: 0.8212 - val_loss: 0.3738 - val_accuracy: 0.8256
Epoch 46/100
1000/1000 [==============================] - 726s 726ms/step - loss: 0.3871 - accuracy: 0.8254 - val_loss: 0.3737 - val_accuracy: 0.8276
Epoch 47/100
1000/1000 [==============================] - 727s 727ms/step - loss: 0.3746 - accuracy: 0.8335 - val

In [ ]:
model1.save(FILENAME+'.model')